In [11]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import plotly.express as px

In [12]:
# load data from previous notebooks
dfAadt = pd.read_csv('intermediate/historic-aadt.csv')
dfLinF = pd.read_csv('results/linear-forecasts.csv')

Create interactive chart with assist from ChatGPT: https://chat.openai.com/share/496c1584-22b6-4c72-9815-395e11718907

In [19]:
# MAKE INTERACTIVE CHART

# Create a new column for segment info
dfLinF['SEGMENT'] = 'Route ' + dfLinF['ROUTE_ID'].str.lstrip('0') + ' - MP ' + dfLinF['FROM_MEASURE'].round(1).astype(str) + ' to ' + dfLinF['TO_MEASURE'].round(1).astype(str)
dfAadt['SEGMENT'] = 'Route ' + dfAadt['ROUTE_ID'].str.lstrip('0') + ' - MP ' + dfAadt['FROM_MEASURE'].round(1).astype(str) + ' to ' + dfAadt['TO_MEASURE'].round(1).astype(str)

# Create a dropdown widget for segment selection
segment_widget = widgets.Select(
    options=dfLinF['SEGMENT'].unique().tolist(),
    description='Segment:',
    disabled=False
)

projgrp_widget = widgets.SelectMultiple(
    options=dfLinF['PROJ_GRP'].unique(),
    description='Proj Groups:',
    disabled=False
)

# Display function
def plot_data(segment, projgrp):

    output.clear_output()
    global firstTime
   
    if firstTime:
        firstTime = False
    else:
        filtereddfLinF = dfLinF[(dfLinF['SEGMENT'] == segment) & (dfLinF['PROJ_GRP'].isin(projgrp))]
        filtereddfAadt = dfAadt[(dfAadt['SEGMENT'] == segment)]
        fig = px.line(filtereddfLinF, x='YEAR', y='LIN_FORECAST', color='PROJ_GRP',
                    title=f'Linear Forecast for {segment}',
                    labels={'LIN_FORECAST': 'AADT', 'YEAR': 'Year'},
                    markers=True) 
        fig.update_traces(line=dict(dash='dash'))
        fig.update_traces(marker=dict(size=20))
    
        # Adding the scatter plot from dfHistoricAADT
        fig.add_scatter(x=filtereddfAadt['YEAR'], y=filtereddfAadt['AADT'], mode='markers', 
                        marker=dict(size=10, color='darkgrey', symbol='circle'), name='Historic AADT')
        
        fig.update_layout(height=500)
        fig.update_xaxes(range=[1980, 2050+2])

        max_LinF = dfLinF[(dfLinF['SEGMENT'] == segment)]['LIN_FORECAST'].max()
        max_Aadt = filtereddfAadt['AADT'].max()
        overall_max = max(max_LinF, max_Aadt)
        fig.update_yaxes(range=[0,overall_max+1000])

        fig.show()
        
# to keep VS CODE from duplicating chart images use global firstTime
firstTime = True

# create output widget to display filtered DataFrame
output = widgets.Output()
hbox = widgets.HBox([segment_widget, projgrp_widget])

# create interactive widget
interactive_output = widgets.interactive_output(plot_data, {'segment': segment_widget, 'projgrp': projgrp_widget})

# Display a markdown heading
display(hbox)
display(interactive_output)
display(output)


Output()

Output()

A few Wastach Front Model Externals

- I-80 MP 98.8 to 101.8
- I-80 MP 136.4 to 139.7
